# Arlington hot spots study

## Explore your data

In [10]:
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
import geopandas as gpd

arlington_bikeshare_july_file = 'arlington_bikeshare_july_agg.geojson'
Map(size_continuous_layer(arlington_bikeshare_july_file,
                          'total_events',
                          'Pickups + Drop Offs'))

## Discover and enrich data

In [4]:
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.data.clients import DataObsClient
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer

# Log in to CARTO 
creds = Credentials('your_username', 'your_api_key')
set_default_credentials(creds)

# Filter out Arlington
do = DataObsClient()
arlington_ct = do.boundaries(region=[-77.17232, 38.827447, -77.032086, 38.93428],
                             boundary='us.census.tiger.census_tract', decode_geom=True)
arlington_df = arlington_ct.dataframe[arlington_ct.dataframe.geom_refs.str.startswith('51013')]

Dataset(arlington_df).upload(table_name='arlington_ct', credentials=creds, if_exists='replace')

# Augmenting it with % of no car households
do.augment('arlington_ct',
           [{"numer_id": "us.census.acs.B08201002", "denom_id": "us.census.acs.B11001001",
             "normalization": "denominated", "geom_id": 'us.census.tiger.census_tract' }],
           how='geom_refs', persist_as='arlington_ct_no_cars')
Map(color_continuous_layer('arlington_ct_no_cars', 'no_cars_2011_2015_by_households', 'No Car Households'))

/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: bf227789-c3e4-4eba-9406-3615b2b5bf20
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))
/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 541817b5-cb99-43f5-b984-32b40efbba89
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


## Analyze your data

In [7]:
from libpysal.weights import Queen
from pysal.explore.esda.moran import Moran_Local
from cartoframes.viz.helpers import color_category_layer

gdf = gpd.GeoDataFrame(Dataset('arlington_ct_no_cars').download(decode_geom=True))
wq = Queen.from_dataframe(gdf)
wq.transform = 'r'
li = Moran_Local(gdf['no_cars_2011_2015_by_households'], wq)
sig = 1 * (li.p_sim < 0.05)
spot_qs = [1, 3, 2, 4] # HH(hotspot), LL(coldspot), LH(doughnut), HL(diamond)
spots = sum([i * (sig * li.q==i) for i in spot_qs])

spot_labels = ['Not significant', 'Hot spot', 'Low outlier', 'Cold spot', 'Hot outlier']
labels = [spot_labels[i] for i in spots]
hmap = ['#E4E4E4','#1785FB', '#F24440', '#12A2B8']
Map([color_category_layer(gdf.assign(cl=labels), 'cl', title='Spot', palette=hmap),
     size_continuous_layer(arlington_bikeshare_july_file, 'total_events', 'Pickups + Drop Offs')])

## Publish and share your maps

In [9]:
# Send results to carto
Dataset(gdf.assign(cl=labels)).upload(table_name='arlington_ct_no_cars', credentials=creds, if_exists='replace')
Dataset(arlington_bikeshare_july_file).upload(table_name='bikeshare_july_agg', credentials=creds, if_exists='replace')

outmap = Map([
    color_category_layer('arlington_ct_no_cars', 'cl', title='Spot', palette=hmap, widget=True),
    size_continuous_layer('bikeshare_july_agg', 'total_events', 'Pickups + Drop Offs', widget=True)
])
publication = outmap.publish('arlington_hot_spots_study')
print(publication['url'])
outmap

/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 97c9bdb7-d165-4c10-9f3c-8f5df5ce72d8
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))
/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 120839ea-ef47-43f8-ba62-9deb0eab3e0c
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


https://team.carto.com/u/cmonteserin/kuviz/8ced83ed-7a24-4778-8fe0-d4242be502be
